# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [7]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [8]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [9]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[:10])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88'], ['Jimi Hendrix', 'Logged In', 'Mohammad', 'M', '1', 'Rodriguez', '239.82975', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Woodstock Inprovisation', '200', '1.54328E+12', '88'], ['Building 429', 'Logged In', 'Mohammad', 'M', '2', 'Rodriguez', '300.61669', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Majesty (LP Version)', '200', '1.54328E+12', '88'], ["The B-52's", 'Logged In', 'Gianna', 'F', '0', 'Jones', '321.54077', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54087E+12', '107', 'Love Shack', '200', '1.54328E+12', '38'], ['Die Mooskirc

In [10]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [11]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
try:
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [12]:
# Create a Keyspace 
try:
    session.execute('''
        CREATE KEYSPACE IF NOT EXISTS "user_activity" WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
        ''')
except Exception as e:
    print(e)

#### Set Keyspace

In [13]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('user_activity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

#### Create queries to ask the following three questions of the data

##### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
##### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
##### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4


#### Assumption: 
Every session of a user (with or without multiple songs) has a unique ID and hence the combination of session_id and item_in_session is unique per row
    
#### Design:
1. Since these are denormalized tables, we will store all the data in the table although we are going to query only for a few values right now - This may help for later queries.
2. Since the query filter is on sessionId and itemInSession, we will use these as the partition key and clustering key respectively.

In [14]:
# build the statement to create the table
query1 = 'CREATE TABLE IF NOT EXISTS session_history ' 
query1 += '(session_id int, item_in_session int, artist text, song text, length float, user_id int, first_name text, last_name text, gender text, level text, location text, '
query1 += 'PRIMARY KEY (session_id, item_in_session))'

try:
    session.execute(query1)
except Exception as e:
    print(e)
        

In [15]:
# write data to table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO session_history (session_id, item_in_session, artist, song, length, user_id, first_name, last_name, gender, level, location) "
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]), int(line[10]), line[1], line[4], line[2], line[6], line[7]))
        except Exception as e:
            print(e)
            
        # debug: print lines matching filter so that we can compare the result of sql
        if int(line[8]) == 338 and int(line[3]) == 4:
            print(line)

['Faithless', 'Ava', 'F', '4', 'Robinson', '495.3073', 'free', 'New Haven-Milford, CT', '338', 'Music Matters (Mark Knight Dub)', '50']


#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
# query for the required data
query = 'SELECT artist, song, length FROM session_history WHERE session_id = 338 AND item_in_session = 4'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print("{}, {}, {}".format(row.artist, row.song, row.length))

Faithless, Music Matters (Mark Knight Dub), 495.30731201171875


In [17]:
# debug: check that the table has 6820 records (minus header row)
query = 'SELECT COUNT(*) FROM session_history'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
print(rows[0])

Row(count=6820)


#### Looking Good!

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Design
1. Since these are denormalized tables, we will store all the data in the table although we are going to query only for a few values right now
2. Since the query filter is on userId and sessionId, we will use these as our partition key and clustering key respectively.
3. However since we also want the data sorted by itemInSession, we will add this as a compound clustering key to preserve the sorting order
4. Since we are having session_id and item_in_session as primary key, we will be able to maintain uniqueness in every row


In [18]:
# build the statement to create the table
query2 = 'CREATE TABLE IF NOT EXISTS user_history ' 
query2 += '(user_id int, session_id int, item_in_session int, artist text, song text, length float, first_name text, last_name text, gender text, level text, location text, '
query2 += 'PRIMARY KEY (user_id, session_id, item_in_session))'

try:
    session.execute(query2)
except Exception as e:
    print(e)      
                    

In [19]:
# write data to table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history (user_id, session_id, item_in_session, artist, song, length, first_name, last_name, gender, level, location) "
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], float(line[5]), line[1], line[4], line[2], line[6], line[7]))
        except Exception as e:
            print(e)
            
        # debug: print lines matching filter so that we can compare the result of sql
        if int(line[10]) == 10 and int(line[8]) == 338:
            print(line)

In [20]:
# query for the required data
query = 'SELECT artist, song, first_name, last_name FROM user_history WHERE user_id = 10 AND session_id = 182'

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print("{}, {}, {}, {}".format(row.artist, row.song, row.first_name, row.last_name))
    

Down To The Bone, Keep On Keepin' On, Sylvie, Cruz
Three Drives, Greece 2000, Sylvie, Cruz
Sebastien Tellier, Kilometer, Sylvie, Cruz
Lonnie Gordon, Catch You Baby (Steve Pitron & Max Sanna Radio Edit), Sylvie, Cruz


In [21]:
# debug: check that the table has 6820 records (minus header row)
query = 'SELECT COUNT(*) FROM user_history'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
print(rows[0])

Row(count=6820)


In [22]:
# debug: checking if it is sorted according to itemInSession
query = 'SELECT item_in_session, artist, song, first_name, last_name FROM user_history WHERE user_id = 10 AND session_id = 182'

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print("{}, {}, {}, {}, {}".format(row.item_in_session, row.artist, row.song, row.first_name, row.last_name))
    

0, Down To The Bone, Keep On Keepin' On, Sylvie, Cruz
1, Three Drives, Greece 2000, Sylvie, Cruz
2, Sebastien Tellier, Kilometer, Sylvie, Cruz
3, Lonnie Gordon, Catch You Baby (Steve Pitron & Max Sanna Radio Edit), Sylvie, Cruz


#### YESSSS!!!

#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


#### Design:

1. In this table I will be storing only the data relevant to the current query that I have in mind - I could store song metadata and user metadata as well
2. The requirement is only to filter by song, so we'll use this as the partition key
3. Since we want the user data that listened to the song, we'll also use user_id as primary key to allow for duplicate song names


In [23]:
# build the statement to create the table

query3 = 'CREATE TABLE IF NOT EXISTS song_history ' 
query3 += '(song_first_letter text, song text, user_id int, first_name text, last_name text, '
query3 += 'PRIMARY KEY (song_first_letter, song, user_id))'

try:
    print(session.execute(query3))
except Exception as e:
    print(e)      
                           

In [24]:
# write data to table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history (song_first_letter, song, user_id, first_name, last_name) "
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        
        try:
            session.execute(query, (line[9][0], line[9], int(line[10]), line[1], line[4])) # song, user_id, first_name, last_name
        except Exception as e:
            print(e)
            
        # debug: print lines matching filter so that we can compare the result of sql
        if line[9] == 'All Hands Against His Own':
            print(line)

['The Black Keys', 'Tegan', 'F', '25', 'Levine', '196.91057', 'paid', 'Portland-South Portland, ME', '611', 'All Hands Against His Own', '80']
['The Black Keys', 'Sara', 'F', '31', 'Johnson', '196.91057', 'paid', 'Winston-Salem, NC', '152', 'All Hands Against His Own', '95']
['The Black Keys', 'Jacqueline', 'F', '50', 'Lynch', '196.91057', 'paid', 'Atlanta-Sandy Springs-Roswell, GA', '559', 'All Hands Against His Own', '29']


In [25]:
# debug: check number of records in table
query = 'SELECT COUNT(*) FROM song_history'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
print(rows[0])

Row(count=6618)


Note: This lower number is expected since the same user might have listened to the same song multiple times

In [26]:
# query for the required data
query = "SELECT first_name, last_name FROM song_history WHERE song_first_letter = 'A' and song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print("{}, {}".format(row.first_name, row.last_name))
   

Jacqueline, Lynch
Tegan, Levine
Sara, Johnson


### Voila!

### Drop the tables before closing out the sessions

In [27]:
## Drop the table before closing out the sessions
try:
    session.execute('DROP TABLE IF EXISTS session_history')
except Exception as e:
    print(e)
    
try:
    session.execute('DROP TABLE IF EXISTS user_history')
except Exception as e:
    print(e)
    
try:
    session.execute('DROP TABLE IF EXISTS song_history')
except Exception as e:
    print(e)
    

### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()